In [5]:
import json
with open('project_ideas.json', 'r') as f:
    project_ideas = json.load(f)
f.close()
f = open('project_names.txt', 'w')
for project in project_ideas:
    f.write(project['project_name'] + '\n')
f.close()


In [2]:
from datasets import load_dataset
swebench = load_dataset('princeton-nlp/SWE-bench', split='test')

/home/mert/.conda/envs/chatdev/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 19008/19008 [00:01<00:00, 12150.51 examples/s]


In [7]:
print(swebench)

Dataset({
    features: ['repo', 'instance_id', 'base_commit', 'patch', 'test_patch', 'problem_statement', 'hints_text', 'created_at', 'version', 'FAIL_TO_PASS', 'PASS_TO_PASS', 'environment_setup_commit'],
    num_rows: 2294
})


In [8]:
print(len(swebench))

2294


In [9]:
swebench[0]

{'repo': 'astropy/astropy',
 'instance_id': 'astropy__astropy-11693',
 'base_commit': '3832210580d516365ddae1a62071001faf94d416',
 'patch': "diff --git a/astropy/wcs/wcsapi/fitswcs.py b/astropy/wcs/wcsapi/fitswcs.py\n--- a/astropy/wcs/wcsapi/fitswcs.py\n+++ b/astropy/wcs/wcsapi/fitswcs.py\n@@ -323,7 +323,17 @@ def pixel_to_world_values(self, *pixel_arrays):\n         return world[0] if self.world_n_dim == 1 else tuple(world)\n \n     def world_to_pixel_values(self, *world_arrays):\n-        pixel = self.all_world2pix(*world_arrays, 0)\n+        # avoid circular import\n+        from astropy.wcs.wcs import NoConvergence\n+        try:\n+            pixel = self.all_world2pix(*world_arrays, 0)\n+        except NoConvergence as e:\n+            warnings.warn(str(e))\n+            # use best_solution contained in the exception and format the same\n+            # way as all_world2pix does (using _array_converter)\n+            pixel = self._array_converter(lambda *args: e.best_solution,\n+ 

In [10]:
swebench[1]

{'repo': 'astropy/astropy',
 'instance_id': 'astropy__astropy-12057',
 'base_commit': 'b6769c18c0881b6d290e543e9334c25043018b3f',
 'patch': 'diff --git a/astropy/nddata/nduncertainty.py b/astropy/nddata/nduncertainty.py\n--- a/astropy/nddata/nduncertainty.py\n+++ b/astropy/nddata/nduncertainty.py\n@@ -395,6 +395,40 @@ def _propagate_multiply(self, other_uncert, result_data, correlation):\n     def _propagate_divide(self, other_uncert, result_data, correlation):\n         return None\n \n+    def represent_as(self, other_uncert):\n+        """Convert this uncertainty to a different uncertainty type.\n+\n+        Parameters\n+        ----------\n+        other_uncert : `NDUncertainty` subclass\n+            The `NDUncertainty` subclass to convert to.\n+\n+        Returns\n+        -------\n+        resulting_uncertainty : `NDUncertainty` instance\n+            An instance of ``other_uncert`` subclass containing the uncertainty\n+            converted to the new uncertainty type.\n+\n+   

In [12]:
import argparse
import logging
import os
import sys

from camel.typing import ModelType


from chatdev.chat_chain import ChatChain

try:
    from openai.types.chat.chat_completion_message_tool_call import ChatCompletionMessageToolCall
    from openai.types.chat.chat_completion_message import FunctionCall

    openai_new_api = True  # new openai api version
except ImportError:
    openai_new_api = False  # old openai api version
    print(
        "Warning: Your OpenAI version is outdated. \n "
        "Please update as specified in requirement.txt. \n "
        "The old API interface is deprecated and will no longer be supported.")

In [13]:
my_dict = {"task_id": "HumanEval/0", "prompt": "from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    \"\"\" Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    \"\"\"\n", "entry_point": "has_close_elements", "canonical_solution": "    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n", "test": "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False\n    assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False\n\n"}


In [14]:
my_dict.keys()

dict_keys(['task_id', 'prompt', 'entry_point', 'canonical_solution', 'test'])

In [15]:
my_dict['test']


"\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False\n    assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False\n\n"

In [16]:
print(my_dict['prompt'])

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """



In [1]:
import pandas as pd

splits = {'test': 'all/test-00000-of-00001.parquet', 'validation': 'all/validation-00000-of-00001.parquet', 'dev': 'all/dev-00000-of-00001.parquet', 'auxiliary_train': 'all/auxiliary_train-00000-of-00001.parquet'}
df_mmlu = pd.read_parquet("hf://datasets/cais/mmlu/" + splits["test"])

/home/mert/.conda/envs/chatdev/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(df_mmlu.head())
print(f"Number of rows: {len(df_mmlu)}")

                                            question           subject  \
0  Find the degree for the given field extension ...  abstract_algebra   
1  Let p = (1, 2, 5, 4)(2, 3) in S_5 . Find the i...  abstract_algebra   
2  Find all zeros in the indicated finite field o...  abstract_algebra   
3  Statement 1 | A factor group of a non-Abelian ...  abstract_algebra   
4  Find the product of the given polynomials in t...  abstract_algebra   

                                             choices  answer  
0                                       [0, 4, 2, 6]       1  
1                                    [8, 2, 24, 120]       2  
2                                   [0, 1, 0,1, 0,4]       3  
3  [True, True, False, False, True, False, False,...       1  
4              [2x^2 + 5, 6x^2 + 4x + 6, 0, x^2 + 1]       1  
Number of rows: 14042


In [3]:
df_mmlu['answer']

0        1
1        2
2        3
3        1
4        1
        ..
14037    0
14038    2
14039    1
14040    1
14041    0
Name: answer, Length: 14042, dtype: int64

In [4]:
def get_prompt(question, choices, subject):
    return f"The following are multiple choice questions (with answers) about {subject}. Think step by" \
             " step and then output the answer in the format of \"The answer is (X)\" at the end. X is the index of the answer (either 0, 1, 2, or 3) \n\n" \
             f"Question: {question}\n\n" \
             f"0. {choices[0]}\n" \
             f"1. {choices[1]}\n" \
             f"2. {choices[2]}\n" \
             f"3. {choices[3]}\n\n" \
             f"Answer: "

In [5]:
import random
import os

# Set random seed for reproducibility
random.seed(42)

# Randomly select 30 rows from the MMLU dataset
sample_indices = random.sample(range(len(df_mmlu)), 30)
sampled_mmlu = df_mmlu.iloc[sample_indices]

# Create a directory to save the prompts if it doesn't exist
os.makedirs("mmlu_prompts", exist_ok=True)

all_prompts = []
# Generate and save prompts for each sampled row
all_answers = []
for i, row in enumerate(sampled_mmlu.itertuples()):
    question = row.question
    choices = [row.choices[0], row.choices[1], row.choices[2], row.choices[3]]
    subject = row.subject
    
    prompt = get_prompt(question, choices, subject)
    
    # # Save the prompt to a file
    # with open(f"mmlu_prompts/prompt_{i}.txt", "w") as f:
    #     f.write(prompt)
    #     # f.write(f"\n\nCorrect answer index: {row.answer}")
    # print(prompt)
    all_answers.append(row.answer)
    all_prompts.append(prompt)

In [14]:
sampled_mmlu

,question,subject,choices,answer
10476,Mast Co. converted from the FIFO method for in...,professional_accounting,"[Lower Lower, Higher Higher, Lower Higher, Hig...",0
1824,What are the dimensions of $\hat{u}^t \hat{u}?,econometrics,"[T x k, T x 1, k x 1, 1 x 1]",3
409,_______ theory can be described as a code of c...,business_ethics,"[Descriptive, Normative, Normative, Instrument...",2
12149,A city imposes a municipal excise tax of $200 ...,professional_law,"[constitutional, as a valid ad valorem tax on ...",1
4506,"Patents, control of resources, economies of sc...",high_school_microeconomics,"[all requirements for price discrimination, re...",3
4012,Which component of a nation's balance of payme...,high_school_macroeconomics,"[The capital account, The official reserves ac...",0
3657,"According to The Federalist Papers, federalism...",high_school_government_and_politics,[It provides a structured environment in which...,1
2286,Keri spent 3 hours on her homework. She spent ...,elementary_mathematics,"[4 subjects, 2 subjects, 6 subjects, 5 subjects]",2
12066,"A teachers union, a nongovernmental entity, se...",professional_law,"[grant relief, because the teachers unionfaile...",2
1679,The surface of Planet Earth loses energy to ou...,conceptual_physics,"[conduction., convection., radiation., radioac...",2


In [16]:
index = 0
counter = 0
# print(all_answers[index])
for j in range(1):
    for index in range(30):
        answers = open(f"/home/mert/mlsys/ChatDev/data_mmlu/mmlu_{index}_{j}.log", "r")
        answers = answers.read()
        if f"({all_answers[index]})" in answers:
            counter += 1
        else:
            print(index)
            print(list(sampled_mmlu['question'])[index])
print(counter/60)

3
A city imposes a municipal excise tax of $200 per year on commercial artists' studios in that city. It also imposes an excise tax of $100 per year on every itinerant commercial figure drawer for the privilege of using the streets and sidewalks. A credit is allowed against this latter tax for any excise tax paid to the city by the drawer or his employer in respect to the maintenance of an artist's studio in the city. A gallery, located in a neighboring state, has been sending two itinerant artists into the city. Their practice is to draw a picture of a pedestrian, ask him to order a finished painting, and collect a payment of $2. 00. The drawing is sent to a studio, which frames it and mails the framed painting to the customer. The neighboring state does not impose a tax on artists' studios. The gallery challenges the constitutionality of the $100 per year excise tax that is imposed upon its itinerant artists in the city. The court will most likely declare this tax
4
Patents, control 

In [37]:
for i,a in enumerate(all_answers):
    print(i,a)

0 0
1 3
2 2
3 1
4 3
5 0
6 1
7 2
8 2
9 2
10 3
11 1
12 0
13 3
14 3
15 0
16 1
17 3
18 3
19 2
20 1
21 2
22 0
23 0
24 1
25 2
26 1
27 2
28 0
29 3


In [40]:
import numpy as np
meta_gpt_answers = [1,1,1,1,1,1,0,1,0,1,1,1,0,1,1,1,1,1,1,0,1,1,0,1,1,0,0,1,1,1]
print(np.mean(meta_gpt_answers))

0.7666666666666667
